In [1]:
import pandas as pd
percentage = 30

## Model Training

In [2]:
df = pd.read_csv('./data/gemstone.csv',skiprows=lambda x: x % int(100/percentage) != 0, nrows=int(1e6))
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
1,5,1.51,Very Good,J,SI1,62.8,58.0,7.34,7.29,4.59,7506
2,8,0.30,Ideal,F,IF,62.0,56.0,4.35,4.37,2.70,886
3,11,0.23,Ideal,E,VVS2,62.0,57.0,3.96,3.98,2.46,505
4,14,1.01,Good,H,SI1,62.6,59.0,6.34,6.40,3.99,4612


In [3]:
df=df.drop(labels=['id'],axis=1)

In [4]:
## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [5]:
Y

,price
0,2772
1,7506
2,886
3,505
4,4612
...,...
64519,789
64520,2307
64521,732
64522,1130


In [6]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [7]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [9]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [10]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [11]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [12]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [13]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.768262,0.631588,-1.158404,0.899607,0.900624,0.950419,-1.141751,0.301094,0.016911
1,0.509974,-0.204609,-0.638894,0.647733,0.701482,0.659395,0.876855,0.301094,0.016911
2,-0.178796,0.724499,-0.119383,-0.062914,-0.031724,0.033694,0.876855,0.918903,-0.648989
3,0.230161,1.746518,-0.119383,0.305903,0.330353,0.499332,-2.151054,0.301094,-1.314888
4,0.251685,0.538677,-0.119383,0.449831,0.393716,0.484781,-0.132448,0.918903,0.016911


In [14]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [15]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [16]:
regression.coef_

array([[ 6450.93800751,  -111.91544122,   -72.53266522, -1902.94873848,
           14.67893627,  -374.01682423,    71.94403148,  -462.18281406,
          655.52215542]])

In [17]:
regression.intercept_

array([3997.68345658])

In [18]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [19]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 992.64741944133
MAE: 669.2104441293936
R2 score 93.70825065992328


Lasso
Model Training Performance
RMSE: 992.8346597423816
MAE: 670.1085892083752
R2 score 93.70587684596792


Ridge
Model Training Performance
RMSE: 992.6338718426124
MAE: 669.2819937418257
R2 score 93.70842239766613


Elasticnet
Model Training Performance
RMSE: 1497.9883464283607
MAE: 1050.298792085335
R2 score 85.67158188748462




In [20]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']